In [ ]:
import cv2
from ultralytics import YOLO
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel
from PyQt5.QtCore import QTimer
from PyQt5.QtGui import QImage, QPixmap
from adbutils import adb
import scrcpy.core as scrcpy
import sys

In [ ]:
phones = ['Pixel 3 XL', 'Pixel 7 Pro']
adb.connect("127.0.0.1:5037")
client = scrcpy.Client(device=adb.device_list()[0])
client.start(threaded=True)
print(client.device_name)
phone_t = phones.index(client.device_name)

In [ ]:
# Load the YOLOv8 model
model = YOLO("yolo_models/yolov8x.pt")

update_timer = 1


def yolo_inference(image):
    # Perform inference
    results = model(image)

    # Get the bounding box data
    bbox_data = results[0].boxes.data

    # Map class IDs to class names
    class_names = results[0].names

    # Check if there are any detections
    if bbox_data.shape[0] > 0:
        for bbox in bbox_data:
            # Extract bounding box coordinates
            x1, y1, x2, y2, conf, cls = bbox.tolist()

            # Convert coordinates to integers
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

            # Draw bounding box on the image
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 5)

            # Draw class name and confidence on the image
            class_name = class_names[int(cls)]
            cv2.putText(image, f'{class_name} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 5)

    return image


def update_ui():
    screen = client.last_frame

    if screen is not None:
        # Perform YOLO inference and get results
        screen_infrenced = yolo_inference(screen)
        
        scale_percent = 50
        width = int(screen_infrenced.shape[1] * scale_percent / 100)
        height = int(screen_infrenced.shape[0] * scale_percent / 100)
        dim = (width, height)
        resized_image = cv2.resize(screen_infrenced, dim, interpolation=cv2.INTER_AREA)
        resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)

        height, width, channel = resized_image.shape
        bytes_per_line = channel * width
        qimage = QImage(resized_image.data, width, height, bytes_per_line, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(qimage)
        screenshot_label.setPixmap(pixmap)

    QTimer.singleShot(update_timer, update_ui)


app = QApplication(sys.argv)
window = QWidget()
window.setWindowTitle("Pokémon Information Display")
layout = QVBoxLayout()


screenshot_label = QLabel()
layout.addWidget(screenshot_label)

window.setLayout(layout)
window.show()

# Start the update loop
QTimer.singleShot(update_timer, update_ui)

app.exec_()